# ESM3 Prediction Notebook

This notebook is intended to be used as a tool for quick and easy protein property prediction using the ESM3 model.


In [ ]:
# @title Input API keys, then hit `Runtime` -> `Run all`
# @markdown Our hosted service that provides access to the full suite of ESM3 models.
# @markdown To utilize the Forge API, users must first agree to the [Terms of Service](https://forge.evolutionaryscale.ai/termsofservice) and generate an access token via the [Forge console](https://forge.evolutionaryscale.ai/console).
# @markdown The console also provides a comprehensive list of models available to each user.

import os

# @markdown ### Authentication
# @markdown Paste your token from the [Forge console](https://forge.evolutionaryscale.ai/console)
forge_token = ""  # @param {type:"string"}
os.environ["ESM_API_KEY"] = forge_token

# @markdown ### Model Selection
# @markdown Enter the model name from the [Forge console page](https://forge.evolutionaryscale.ai/console) that you would like to use:
model_name = "esm3-medium-2024-08"  # @param {type:"string"}

# markdown ### Sequence
# @markdown Please use '|' to delimit a multimer sequence.
sequence = "MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKPLSVSYDQATSLRILNNGHAFNVEFDDSQDKAVLKGGPLDGTYRLIQFHFHWGSLDGQGSEHTVDKKKYAAELHLVHWNTKYGDFGKAVQQPDGLAVLGIFLKVGSAKPGLQKVVDVLDSIKTKGKSADFTNFDPRGLLPESLDYWTYPGSLTTPPLLECVTWIVLKEPISVSSEQVLKFRKLNFNGEGEPEELMVDNWRPAQPLKNRQIKASFK"  # @param {type:"string"}

In [ ]:
# @title Install dependencies
import os

os.system("pip install git+https://github.com/evolutionaryscale/esm")
os.system("pip install pydssp pygtrie dna-features-viewer py3dmol ipywidgets")

In [ ]:
# @title Run Prediction and Display Results
from esm.sdk.api import ESMProtein, ESMProteinError, GenerationConfig
from esm.widgets.components.results_visualizer import create_results_visualizer
from esm.widgets.utils.clients import get_forge_client
from ipywidgets import widgets

# Initialize client
client = get_forge_client(model_name)

# Create protein object
protein = ESMProtein(sequence=sequence)

# Predict all tracks
tracks = ["structure", "secondary_structure", "sasa", "function"]

output = widgets.Output()
display(output)
with output:
    print("Starting predictions...")

    for track in tracks:
        print(f"Predicting {track}...")
        protein = client.generate(
            protein, config=GenerationConfig(track=track, temperature=0.01)
        )
        if isinstance(protein, ESMProteinError):
            raise RuntimeError(f"Error: {str(protein)}")

    # Create result visualizers
    structure_results = create_results_visualizer(
        modality="structure", samples=[protein], items_per_page=1, include_title=False
    )

    secondary_structure_results = create_results_visualizer(
        modality="secondary_structure",
        samples=[protein],
        items_per_page=1,
        include_title=False,
    )

    sasa_results = create_results_visualizer(
        modality="sasa", samples=[protein], items_per_page=1, include_title=False
    )

    function_results = create_results_visualizer(
        modality="function", samples=[protein], items_per_page=1, include_title=False
    )

    output.clear_output(wait=True)

    # Create tabbed interface
    results_ui = widgets.Tab(
        children=[
            structure_results,
            secondary_structure_results,
            sasa_results,
            function_results,
        ]
    )
    results_ui.set_title(0, "Structure")
    results_ui.set_title(1, "Secondary Structure")
    results_ui.set_title(2, "SASA")
    results_ui.set_title(3, "Function")
    display(results_ui)